In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier


In [15]:
df=pd.read_csv("6th_student_ds.csv")
df

,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer,Interest,Recommended Career
0,39,46,54,35,71,64,74,53,28,20,74,67,Geography,Archaeologist
1,54,56,69,54,72,27,22,79,47,79,65,25,Environmental Studies,Environmental Scientist
2,59,68,22,77,60,36,74,71,55,79,76,53,Environmental Studies,Environmental Scientist
3,31,61,40,25,26,61,78,22,38,76,24,60,Geography,Geotechnical Engineer
4,52,25,22,21,59,35,44,53,62,45,24,76,Computer,Software Developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,71,79,27,23,40,54,20,49,56,59,31,73,Urdu,Literature and Education
9996,57,56,77,72,23,22,43,43,25,23,41,21,Hindi,Media and Journalism
9997,22,75,69,33,25,74,35,44,80,30,53,49,Sports,Sports Coach
9998,26,20,72,58,25,58,46,46,80,35,75,51,Sports,Professional Athlete


In [16]:
df.dtypes

Marathi                   int64
Urdu                      int64
Hindi                     int64
English                   int64
History                   int64
Science                   int64
Geography                 int64
Drawing                   int64
Sports                    int64
Environmental Studies     int64
Math                      int64
Computer                  int64
Interest                 object
Recommended Career       object
dtype: object

In [17]:
df2 = df.drop(['Interest', 'Recommended Career'], axis=1)
df2

,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer
0,39,46,54,35,71,64,74,53,28,20,74,67
1,54,56,69,54,72,27,22,79,47,79,65,25
2,59,68,22,77,60,36,74,71,55,79,76,53
3,31,61,40,25,26,61,78,22,38,76,24,60
4,52,25,22,21,59,35,44,53,62,45,24,76
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,71,79,27,23,40,54,20,49,56,59,31,73
9996,57,56,77,72,23,22,43,43,25,23,41,21
9997,22,75,69,33,25,74,35,44,80,30,53,49
9998,26,20,72,58,25,58,46,46,80,35,75,51


In [18]:
df2.insert(0, 'Unique_ID', [f"stud_{i}" for i in range(1, len(df2) + 1)])
df2

,Unique_ID,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer
0,stud_1,39,46,54,35,71,64,74,53,28,20,74,67
1,stud_2,54,56,69,54,72,27,22,79,47,79,65,25
2,stud_3,59,68,22,77,60,36,74,71,55,79,76,53
3,stud_4,31,61,40,25,26,61,78,22,38,76,24,60
4,stud_5,52,25,22,21,59,35,44,53,62,45,24,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,71,79,27,23,40,54,20,49,56,59,31,73
9996,stud_9997,57,56,77,72,23,22,43,43,25,23,41,21
9997,stud_9998,22,75,69,33,25,74,35,44,80,30,53,49
9998,stud_9999,26,20,72,58,25,58,46,46,80,35,75,51


In [19]:
# df2['Interest'] = df2.idxmax(axis=1)
# df2

In [20]:
# Ensure all subject columns are numeric
df2.iloc[:, 1:] = df2.iloc[:, 1:].apply(pd.to_numeric, errors="coerce")

# Function to find top and second-top subjects excluding Unique_ID
def find_top_interests(row):
    # Exclude the Unique_ID column
    subject_marks = {subject: mark for subject, mark in row.items() if subject != "Unique_ID"}
    
    # Find the highest and second-highest marks
    unique_marks = sorted(set(subject_marks.values()), reverse=True)
    if len(unique_marks) >= 2:
        top_marks = unique_marks[:2]  # Top two unique marks
    elif len(unique_marks) == 1:
        top_marks = unique_marks  # Only one unique mark
    else:
        top_marks = []  # No marks present
    
    # Collect all subjects with marks in top_marks
    top_subjects = [subject for subject, mark in subject_marks.items() if mark in top_marks]
    return ", ".join(top_subjects)

# Apply the function to find the Interest column
df2["Interest"] = df2.apply(find_top_interests, axis=1)
df2
file_path = "Latest6th.csv"  # Specify your desired file path
df2.to_csv(file_path, index=False)  # Save the DataFrame without the index
print(f"CSV file saved")

CSV file saved


In [175]:
# Career map dictionary
career_map = {
    "Marathi": ["Media and Journalism", "Entertainment and Arts", "Literature and Education", "Government and Administrative Services"],
    "Urdu": ["Media and Journalism", "Entertainment and Arts", "Literature and Education", "Government and Administrative Services"],
    "Hindi": ["Media and Journalism", "Entertainment and Arts", "Literature and Education", "Government and Administrative Services"],
    "English": ["Media and Journalism", "Entertainment and Arts", "Literature and Education", "Government and Administrative Services", "Corporate and Business Communication"],
    "History": ["Historian", "Archaeologist", "Museum Professional", "Cultural Heritage Manager", "Genealogist", "Documentary Filmmaker"],
    "Science": ["Healthcare and Medicine", "Research and Development", "Agriculture and Food Sciences", "Education and Academia", "Allied Healthcare Professions", "Interdisciplinary Fields"],
    "Geography": ["Urban and Regional Planner", "Environmental Consultant", "Cartographer", "Geospatial Analyst", "Meteorologist", "Geologist", "Disaster Management Specialist", "Archaeologist", "Geographic Information Systems (GIS) Specialist", "Geotechnical Engineer"],
    "Drawing": ["Fine Artist", "Graphic Designer", "Concept Artist", "Animator", "Fashion Designer", "Comic Artist/Cartoonist", "Product Designer", "Architect", "Storyboard Artist", "Set Designer/Illustrator for Theatre or Film", "Visual Development Artist", "3D Artist"],
    "Sports": ["Professional Athlete", "Sports Coach", "Sports Nutritionist/Dietitian", "Physical Therapist", "Fitness Trainer"],
    "Environmental Studies": ["Environmental Scientist"],
    "Math": ["Engineering", "Research and Development (R&D)", "Mathematics and Actuarial Science", "Architecture", "Aviation and Aerospace", "Finance and Investment"],
    "Computer": ["Software Developer", "Artificial Intelligence (AI) / Machine Learning Engineer"]
}

# Function to map interests (comma-separated string) to careers
def map_to_careers_from_interests(interests):
    # Split the comma-separated string into a list of subjects
    interest_list = interests.split(', ')
    careers = []
    for subject in interest_list:
        careers.extend(career_map.get(subject, []))
    return ', '.join(careers)

# Apply the mapping function based on the 'Interest' column
df2['Recommended Career'] = df2['Interest'].apply(map_to_careers_from_interests)
df2

,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer,Interest,Recommended Career
0,39,46,54,35,71,64,74,53,28,20,74,67,"History, Geography, Math","Historian, Archaeologist, Museum Professional,..."
1,54,56,69,54,72,27,22,79,47,79,65,25,"History, Drawing, Environmental Studies","Historian, Archaeologist, Museum Professional,..."
2,59,68,22,77,60,36,74,71,55,79,76,53,"English, Environmental Studies","Media and Journalism, Entertainment and Arts, ..."
3,31,61,40,25,26,61,78,22,38,76,24,60,"Geography, Environmental Studies","Urban and Regional Planner, Environmental Cons..."
4,52,25,22,21,59,35,44,53,62,45,24,76,"Sports, Computer","Professional Athlete, Sports Coach, Sports Nut..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,71,79,27,23,40,54,20,49,56,59,31,73,"Urdu, Computer","Media and Journalism, Entertainment and Arts, ..."
9996,57,56,77,72,23,22,43,43,25,23,41,21,"Hindi, English","Media and Journalism, Entertainment and Arts, ..."
9997,22,75,69,33,25,74,35,44,80,30,53,49,"Urdu, Sports","Media and Journalism, Entertainment and Arts, ..."
9998,26,20,72,58,25,58,46,46,80,35,75,51,"Sports, Math","Professional Athlete, Sports Coach, Sports Nut..."


In [153]:
df2.describe()

,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,49.925000,50.046800,50.093500,49.904400,50.159300,50.074600,49.793800,50.070800,50.093300,50.007400,50.11360,50.284500
std,17.455963,17.607629,17.604776,17.700666,17.590812,17.563669,17.600644,17.650664,17.546076,17.684649,17.73908,17.619064
min,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000
25%,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.00000,35.000000
50%,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000
75%,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,66.000000,66.00000,66.000000
max,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.00000,80.000000


In [154]:
#5th std
df3=pd.read_csv("New5th.csv")
df3

,Marathi,Urdu,Hindi,English,History,Science,Geography,Drawing,Sports,Environmental Studies,Math,Computer,Interest,Recommended Career
0,25,33,33,83,63,13,20,92,99,45,99,71,"Sports, Math","Professional Athlete, Architecture, Sports Nut..."
1,52,58,64,24,34,20,43,89,89,20,56,31,"Drawing, Sports","3D Artist, Professional Athlete, Product Desig..."
2,76,24,94,29,91,10,28,86,52,14,12,62,Hindi,"Media and Journalism, Literature and Education..."
3,41,70,81,98,74,13,40,56,25,36,35,64,English,"Entertainment and Arts, Corporate and Business..."
4,88,54,49,78,2,11,90,80,89,37,64,91,Computer,"Software Developer, Artificial Intelligence (A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,40,6,62,9,37,38,85,66,4,2,70,34,Geography,"Geospatial Analyst, Archaeologist, Urban and R..."
99996,29,58,39,38,2,53,97,49,10,48,6,80,Geography,"Geospatial Analyst, Archaeologist, Urban and R..."
99997,9,53,35,95,48,47,48,67,11,51,59,36,English,"Entertainment and Arts, Corporate and Business..."
99998,42,76,49,78,87,75,38,13,81,96,64,42,Environmental Studies,Environmental Scientist


In [161]:
# Function to calculate interests based on highest marks
def calculate_interest(row):
    numeric_row = row.apply(pd.to_numeric, errors='coerce')  # Ensure numeric conversion
    max_marks = numeric_row.max()
    subjects = [subject for subject in numeric_row.index if numeric_row[subject] == max_marks]
    return ', '.join(subjects)

# Determine interests for each dataset
df3['Interest'] = df3.apply(calculate_interest, axis=1)
df2['Interest'] = df2.apply(calculate_interest, axis=1)

# Merge the datasets without Student column
merged_df = pd.concat([df3[['Interest']].rename(columns={'Interest': '5th'}), 
                       df2[['Interest']].rename(columns={'Interest': '6th'})], axis=1)

# Function to determine combined interests
def combine_interests(row):
    interests_5th = set(row['5th'].split(', ')) if pd.notna(row['5th']) else set()
    interests_6th = set(row['6th'].split(', ')) if pd.notna(row['6th']) else set()
    common_interests = interests_5th.intersection(interests_6th)

    if common_interests:
        return ', '.join(sorted(common_interests))
    else:
        all_interests = interests_5th.union(interests_6th)
        return ', '.join(sorted(all_interests))

# Apply the function to determine final interest
merged_df['Final_Interest'] = merged_df.apply(combine_interests, axis=1)

# Final dataset with combined interests
final_df = merged_df[['Final_Interest']]

final_df


,Final_Interest
0,Math
1,Drawing
2,"Environmental Studies, Hindi"
3,"English, Geography"
4,Computer
...,...
99995,Geography
99996,Geography
99997,English
99998,Environmental Studies


In [155]:
df2.info

<bound method DataFrame.info of       Marathi  Urdu  Hindi  English  History  Science  Geography  Drawing  \
0          39    46     54       35       71       64         74       53   
1          54    56     69       54       72       27         22       79   
2          59    68     22       77       60       36         74       71   
3          31    61     40       25       26       61         78       22   
4          52    25     22       21       59       35         44       53   
...       ...   ...    ...      ...      ...      ...        ...      ...   
9995       71    79     27       23       40       54         20       49   
9996       57    56     77       72       23       22         43       43   
9997       22    75     69       33       25       74         35       44   
9998       26    20     72       58       25       58         46       46   
9999       58    41     53       24       32       63         60       21   

      Sports  Environmental Studies  Math  

In [156]:
df2.isnull().sum()

Marathi                  0
Urdu                     0
Hindi                    0
English                  0
History                  0
Science                  0
Geography                0
Drawing                  0
Sports                   0
Environmental Studies    0
Math                     0
Computer                 0
Interest                 0
Recommended Career       0
dtype: int64

In [16]:
max_marks = df2['Marathi'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Marathi'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Marathi: {count_max_marks}")

Number of students with the maximum marks in Marathi: 157


In [17]:
max_marks = df2['Urdu'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Urdu'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Urdu: {count_max_marks}")

Number of students with the maximum marks in Urdu: 155


In [18]:
max_marks = df2['Hindi'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Hindi'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Hindi: {count_max_marks}")

Number of students with the maximum marks in Hindi: 144


In [19]:
max_marks = df2['English'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['English'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in English: {count_max_marks}")

Number of students with the maximum marks in English: 160


In [20]:
max_marks = df2['History'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['History'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in History: {count_max_marks}")

Number of students with the maximum marks in History: 146


In [21]:
max_marks = df2['Science'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Science'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Science: {count_max_marks}")

Number of students with the maximum marks in Science: 135


In [22]:
max_marks = df2['Geography'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Geography'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Geography: {count_max_marks}")

Number of students with the maximum marks in Geography: 143


In [23]:
max_marks = df2['Drawing'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Drawing'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Drawing: {count_max_marks}")

Number of students with the maximum marks in Drawing: 160


In [24]:
max_marks = df2['Sports'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Sports'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Sports: {count_max_marks}")

Number of students with the maximum marks in Sports: 168


In [25]:
max_marks = df2['Environmental Studies'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Environmental Studies'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Environmental Studies: {count_max_marks}")

Number of students with the maximum marks in Environmental Studies: 167


In [26]:
max_marks = df2['Computer'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Computer'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Computer: {count_max_marks}")

Number of students with the maximum marks in Computer: 148


In [27]:
max_marks = df2['Math'].max()

# Count the number of students with the maximum marks
count_max_marks = df2[df2['Math'] == max_marks].shape[0]

print(f"Number of students with the maximum marks in Math: {count_max_marks}")

Number of students with the maximum marks in Math: 165


In [28]:
#x = df.drop(['Recommended Career'], axis=1)

In [29]:
#x=x.values

In [561]:
# # Extract the subject of interest based on max marks
# subjects = ["Marathi", "Urdu", "Hindi", "English", "History", "Science", 
#             "Geography", "Drawing", "Sports", "Environmental Studies", "Math", "Computer"]
# df['Interest'] = df[subjects].idxmax(axis=1)  # Identify the subject with max marks

# # One-hot encode the 'Interest' column
# encoder = OneHotEncoder()
# interest_encoded = encoder.fit_transform(df[['Interest']]).toarray()

In [562]:
# Elbow method to find the optimal number of clusters
# wcss = []
# for i in range(1, 20):  # Test 1 to 10 clusters
#     kmeans = KMeans(n_clusters=i, random_state=42)
#     kmeans.fit(interest_encoded)
#     wcss.append(kmeans.inertia_)

# # Plot the Elbow Curve
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, 20), wcss, marker='o', linestyle='--')
# plt.title('Elbow Method for Optimal Clusters')
# plt.xlabel('Number of Clusters')
# plt.ylabel('WCSS')
# plt.xticks(range(1, 20))
# plt.grid()
# plt.show()

In [563]:
# optimal_clusters = 12  # Replace with the actual number determined from the elbow curve
# kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
# y_means['Cluster']= kmeans.fit_predict(interest_encoded)

# # Analyze and save results
# print(df[['Interest', 'Cluster']].head(12))
# df.to_csv("clustered_stu.csv", index=False)

In [564]:
# a=df['Recommended Career']
# y_means = pd.DataFrame(y_means)
# #Converts the NumPy array y_means (containing cluster assignments) into a pandas DataFrame.

# z = pd.concat([y_means, a],axis = 1)
# #Concatenates the two DataFrames (y_means and a) along the columns (axis=1), combining the cluster assignments with the 
#     #original labels.

# z = z.rename(columns = {0: 'cluster'})

In [565]:
# # Group the data by clusters
# clusters = df.groupby('Cluster')

# # View data for each cluster
# for cluster_id, cluster_data in clusters:
#     print(f"Cluster {cluster_id}:\n")
#     print(cluster_data.head())  # Display the first few rows of the cluster
#     print("\n" + "-"*50 + "\n")


In [566]:
#x = df.drop(['Recommended Career'], axis=1)

In [30]:
# Features and Target
X = df2[['Interest']]
y = df2['Recommended Career']


In [31]:
le = LabelEncoder()
df2['Recommended Career'] = le.fit_transform(df2['Recommended Career'])  # Fit label encoder on the entire dataset
y = df2['Recommended Career']

In [32]:
X = pd.get_dummies(X)

In [33]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [34]:
ensemble_model = VotingClassifier(estimators=[('rf', RandomForestClassifier()), ('xgb', XGBClassifier())], voting='soft')
ensemble_model.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))],
                 voting='soft')

In [35]:
# Train a model (Random Forest in this case)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [36]:
# Predictions
y_pred = model.predict(X_test)

In [37]:
# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.998
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       267
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00       234
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00       260
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00        10
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00        10
          13       1.00      1.00      1.00         6
          16       0.40      1.00      0.57         4
          17       1.00      1.00      1.00         6
          18       0.00      0.00      0.00         1
          20       1.00      1.00      1.00         7
          24       1.00      1.00      1.00       269
          25       1.00      1.00      1.00         5
          2

C:\Users\Abhishek Talole\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek Talole\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek Talole\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [43]:
# Test with a new student
new_student = pd.DataFrame({
    'Interest': ['Math','Sports']
   
    
})
new_student = pd.get_dummies(new_student)
new_student = new_student.reindex(columns=X.columns, fill_value=0)

In [44]:
predicted_career = model.predict(new_student)
predicted_output=le.inverse_transform(predicted_career)
print("Predicted Career:",predicted_output)

Predicted Career: ['Engineering, Research and Development (R&D), Aviation and Aerospace, Architecture, Finance and Investment, Mathematics and Actuarial Science'
 'Sports Nutritionist/Dietitian, Fitness Trainer, Sports Coach, Physical Therapist, Professional Athlete']
